In [2]:
import pandas as pd
import numpy as np

In [294]:
sc_st_sections = pd.read_csv('SC_ST_sections.csv')
poa_offenses = pd.read_csv('POA_offenses.csv')

set_ipc_sections = set(sc_st_sections.IPC.tolist())
set_ipc_sections.add('ipc')

# sc_st_sections = sc_st_sections.iloc[:300]
sc_st_sections.raw_no = sc_st_sections.raw_no.astype(str)

clauses_dict = poa_offenses.loc[:,['Sub-sub-sectoin', 'Alternate name']].set_index('Sub-sub-sectoin').to_dict()['Alternate name']
clauses_dict['da'] = '18'
clauses_dict['dha'] = '19'

In [295]:
sc_st_sections.head()

,num,raw_no,sec,sub_sec,clause,IPC,clean_further,other
0,9940,3,3.0,1.0,NaN,NaN,NaN,NaN
1,5033,1,3.0,1.0,NaN,NaN,NaN,NaN
2,3427,3(1)10,3.0,1.0,10,NaN,NaN,NaN
3,2962,3(1)(10),3.0,1.0,10,NaN,NaN,NaN
4,2642,3(1)X,3.0,1.0,10,NaN,NaN,NaN


In [345]:
def clean_act_name(name):
    clean_name_set = []
    name = str(name)
    for ipc_section in set_ipc_sections:
        name = name.replace(str(ipc_section),'')
    name = name.replace('(', ',').replace(')', ',')
    name = name.replace('-', ',')
    name = name.replace(' ', ',')
    name = name.split(',')
    name = list(filter(lambda x: x != '', name))
    if not name:
        return np.nan
    count = 0
    while count<5 and name:
        count+=1
        try:
            clean_name = ['3', '1', np.nan]
#             print(name, clean_name)
            if name[0]=='3':
                clean_name[0] = name.pop(0)
            if name[0]=='1' or name[0]=='2':
                clean_name[1]=name.pop(0)
                
            try:
                if roman.fromRoman(name[0])<30:
                    clean_name[2] = roman.fromRoman(name[0])
                    name.pop(0)
            except:
                pass
            
            try:
                if name[0].lower() in clauses_dict.keys():
                    clean_name[2] = clauses_dict[name.pop(0).lower()]
                    clean_name_set.append(clean_name)
                    continue
            except AttributeError:
                pass
            
            try:
                if name[0] in clauses_dict.values():
                    clean_name[2] = name.pop(0)
                    clean_name_set.append(clean_name)
                    continue
            except AttributeError:
                pass
            
            try:
                if ord(name[0])>2324 and ord(name[0])<2360:
                    clean_name[2]= ord(name.pop(0))-2324
            except TypeError:
                pass
            
            if clean_name not in clean_name_set:
                clean_name_set.append(clean_name)
#             print(name, clean_name)
        except IndexError:
            clean_name_set.append(clean_name)
            return clean_name_set
    
    if len(name)==1:
        while count<10 and name:
            count+=1
            try:
                name = name[0]
                clean_name = ['3', np.nan, np.nan]
                if name[0]=='3':
                    name = name[1:]
                if name[0]=='1' or name[0]=='2':
                    clean_name[1]=name[0]
                    name = name[1:]
                if name[:2] in clauses_dict.values():
                    clean_name[2] = name[:2]
                    name = name[2:]
                if name[:2] in clauses_dict.keys():
                    clean_name[2] = clauses_dict[name[:2]]
                    name = name[2:]
                if name[:1] in clauses_dict.values():
                    clean_name[2] = name[:1]
                    name = name[1:]
                if name[:1] in clauses_dict.keys():
                    clean_name[2] = clauses_dict[name[:1]]
                    name = name[1:]
                    
#                 for clause in clauses_dict.values():
#                     if name.startswith(clause):
#                         clean_name[2] = clause
#                         name = name.replace(clause, '', 1)
#                 for clause in clauses_dict.keys():
#                     if name.startswith(clause):
#                         clean_name[2] = clauses_dict[clause]
#                         name = name.replace(clause, '', 1)

                if clean_name not in clean_name_set:
                    clean_name_set.append(clean_name)   
            except IndexError:
                clean_name_set.append(clean_name)
                return clean_name_set
        
    if len(clean_name_set)>1:
        return [clean_name for clean_name in clean_name_set if str(clean_name[2]) != 'nan']
    return clean_name_set
    
# print(clean_act_name('3-1-10'))    
sc_st_sections['clean_act'] = sc_st_sections.raw_no.apply(clean_act_name)   
sc_st_sections.to_csv('sc_st_sections_new.csv')

In [343]:
sc_st_sections.head(50)

,num,raw_no,sec,sub_sec,clause,IPC,clean_further,other,clean_act
0,9940,3,3.0,1.0,NaN,NaN,NaN,NaN,"[[3, 1, nan]]"
1,5033,1,3.0,1.0,NaN,NaN,NaN,NaN,"[[3, 1, nan]]"
2,3427,3(1)10,3.0,1.0,10,NaN,NaN,NaN,"[[3, 1, 10]]"
3,2962,3(1)(10),3.0,1.0,10,NaN,NaN,NaN,"[[3, 1, 10]]"
4,2642,3(1)X,3.0,1.0,10,NaN,NaN,NaN,"[[3, 1, 10]]"
5,1978,3110,3.0,1.0,10,NaN,NaN,NaN,"[[3, 1, 10]]"
6,1893,10,3.0,1.0,10,NaN,NaN,NaN,"[[3, 1, 10]]"
7,1764,3(1),3.0,1.0,NaN,NaN,NaN,NaN,"[[3, 1, nan]]"
8,1710,3(1)(X),3.0,1.0,10,NaN,NaN,NaN,"[[3, 1, 10]]"
9,1549,2,NaN,NaN,NaN,NaN,NaN,NaN,"[[3, 2, nan]]"


In [ ]:
def parse_string(name):
    clean_name = [np.nan, np.nan, np.nan]
    if not name:
        return None
    
    


def clean_act_name(name):
    clean_name_set = []
    name = str(name)
    for ipc_section in set_ipc_sections:
        name = name.replace(str(ipc_section),'')
    name = name.replace('(', ',').replace(')', ',')
    name = name.replace('-', ',')
    name = name.replace(' ', ',')
    name = name.split(',')
    name = list(filter(lambda x: x != '', name))
    if not name:
        return np.nan
    count = 0
    while count<5 and name:
        count+=1
        try:
            clean_name = [np.nan, np.nan, np.nan]
#             print(name, clean_name)
            if name[0]=='3':
                clean_name[0] = name.pop(0)
            if name[0]=='1' or name[0]=='2':
                clean_name[1] = name.pop(0)
            try:
                clean_name[2] = roman.fromRoman(name[0])
                name.pop(0)
            except:
                pass
#             name=[item.lower() for item in name]
            if name[0] in clauses_dict.keys():
                clean_name[2] = clauses_dict[name.pop(0)]
                clean_name_set.append(clean_name)
                continue
            if name[0] in clauses_dict.values():
                clean_name[2] = name.pop(0)
                clean_name_set.append(clean_name)
                continue
            try:
                if ord(name[0])>2324 and ord(name[0])<2360:
                    clean_name[2]= ord(name.pop(0))-2324
            except TypeError:
                pass
            if clean_name not in clean_name_set:
                clean_name_set.append(clean_name)
#             print(name, clean_name)
        except IndexError:
            clean_name_set.append(clean_name)
            return clean_name_set
    return clean_name_set
    
# print(clean_act_name('3-1-10'))    
sc_st_sections['clean_act'] = sc_st_sections.raw_no.apply(clean_act_name)   

In [327]:
a = '31103111'
a.replace('11', '', 1)

'303111'

In [341]:
list(reversed(list(clauses_dict.keys())))
list(clauses_dict.values())

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '12',
 '10',
 '11',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '5',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '1',
 '2',
 '3',
 '4',
 '5a',
 '6',
 '7',
 '18',
 '19']

In [339]:
)

SyntaxError: invalid syntax (<ipython-input-339-a35077a93b18>, line 1)

In [346]:
import os
os.getcwd()

'C:\\Users\\Nick\\Desktop\\Chicago\\DIME\\kanoon'